# Running GTWR model on COVID data with pre-defined waves information

- Prerequisite python libraies: pandas, geopandas, numpy, gtwr
- Input data: Covid cases aggregated by ZCTA at weekly level
- Output data: 
  1. *model_diagnosis* : diagnosis metrics for the fitted GTWR model including **<'AIC','BIC','R^2', Adjusted R^2'>**
  1. *model_coeff*     : model outputs including **<coefficient, t-statistic, standard residual>**
  1. *model_params*    : parameters that were used to fit the GTWR model 
  
---

### Analysis workflow:

1. Configuration: define waves (start/end date)
1. Configuration: define global parameters
1. Data pre-processing: prepare shapefile of the study area, process case data, prepare variables
1. Finalize model input data: standardize varaibels 
1. **[Optional]** use Golden Search to find optimized parameters (bandwidth and patio-temporal scale)
1. Fit GTWR model 
1. Export model output and model settings to .csv files

In [2]:
### Import required packages
import pandas as pd

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

import numpy as np

In [3]:
### If not installed yet, install gtwr library on the fly
!pip install gtwr

### Import gtwr library
from gtwr import model
from gtwr.model import GTWR

In [12]:
### Use these function to check model parameters 

# help(model.GTWR)
# help(GTWR)
# dir(GTWR)

--- 
# Step 0: Overal Configurations

In [ ]:
### Create a ditionary that holds some configurations of waves, starting/ending date

wave_info = {
    'wave1' : {
        'month_range': ['2020May','2020Sep'],
        'data_range_start':'2020-05-01',
        'data_range_end':'2020-09-30',
    },
    'wave2' : {
        'month_range': ['2020Oct','2021Mar'],
        'data_range_start':'2020-10-01',
        'data_range_end':'2021-03-31',
    },
    'wave3' : {
        'month_range': ['2021Jun','2021Nov'],
        'data_range_start':'2021-06-01',
        'data_range_end':'2021-11-30',
    },
    'wave4' : {
        'month_range': ['2021Dec','2022Mar'],
        'data_range_start':'2021-12-01',
        'data_range_end':'2022-03-31',
    },
    'wave_test' : {
        'month_range': ['2020Apr','2020Apr'],
        'data_range_start':'2020-04-01',
        'data_range_end':'2020-04-30',
    },
    'wave_test_long' : {
        'month_range': ['2020Apr','2021Apr'],
        'data_range_start':'2020-04-01',
        'data_range_end':'2021-04-30',
    },
    'wave_test_18month' : {
        'month_range': ['2020Apr','2021Oct'],
        'data_range_start':'2020-04-01',
        'data_range_end':'2021-10-31',
    },
     'wave_test_24month' : {
        'month_range': ['2020Apr','2022Apr'],
        'data_range_start':'2020-04-01',
        'data_range_end':'2022-04-30',
    },
}

In [ ]:
### ------------------------------------------------------------------------------------------------------------
### ! Important !
### Model Specification : CHANGE these two as you run different models for waves/convariates combinations 
### ------------------------------------------------------------------------------------------------------------

# This wave name correspondes to the name defined in the above wave_info ditionary  ['wave1', 'wave2', ... , 'wave_test_24month'] 
wave = 'wave_test_24month'   

# This index correspondes to the order of the covariates in the config['X_cov'] setting below: ['Hispanic','Crowding','NonHisBlk','EmployService','NoHighSchool']
# INDEX == 0 means the covariate is 'Hispanic'
X_cov_run_index = 0       


### ---------------------------------
### Global Parameters
### ---------------------------------
waveID = wave
X_cov_run = X_cov_run_index

config = {
    'data_path' : "C:/Users/jyang/Documents/Covid_MUAP_GTWR/FullData/",    # 
    'case_data' : "C:/Users/jyang/Documents/Covid_MUAP_GTWR/FullData/case_var_all0915.csv",
    'subset' : True,          # Set to True to indicate that the model is running either with (a) subset region or (b) subset time frame
    'subset_region' : False,  # Set to True if the model is running either with [subset region]
    'subset_date' : True,     # Set to True if the model is running either with [subset time frame]
    'region' : 'SD',          # If running model for a [subset region], name of the region.  Example: SD/LA/SF...etc
    'month_range' :      wave_info[waveID]['month_range'],       # Analysis month range, format: ['2020May','2020Sep'], 
    'data_range_start' : wave_info[waveID]['data_range_start'],  # Analysis start date, format: '2020-05-01', 
    'data_range_end' :   wave_info[waveID]['data_range_end'],    # Analysis end date, format: '2020-09-30',
    'study_area_Full' : "C:/Users/jyang/Documents/Covid_MUAP_GTWR/FullData/ZCTA_Update.shp",   # shapefile of the study area [full region]
    'study_area_SD' : "C:/Users/jyang/Documents/Covid_MUAP_GTWR/FullData/ZCTA_Update_SDCounty_4326.shp",  # shapefile of the study area [subset region - SD]
    'study_area_LA' : "C:/Users/jyang/Documents/Covid_MUAP_GTWR/FullData/ZCTA_Update_SDCounty_4326.shp",  # shapefile of the study area [subset region - LA]
    'study_area_SF' : "C:/Users/jyang/Documents/Covid_MUAP_GTWR/FullData/ZCTA_Update_SDCounty_4326.shp",  # shapefile of the study area [subset region - SF]
    'bw_type' : 'fixed',     # indicate if the model uses 'fixed' or 'adaptive' bandwidth 
    'use_goldenSearch': True,  # indicate if the workflow includes a step of finding optimized model parameters (bw and tau) using Golden Search 
    'bw' : 0.4,    # bandwidth value consisting of either a distance or N nearest neighbors 
    'tau':  4000,  # spatio-temporal scale
    'tau_min': 3000,  # tau's lower-bound during the golden search
    'tau_max': 7000,  # tau's upper-bound during the golden search
    'sensiTest_tau': True,  # Set this TRUE if running sensitivity test on bw/tau, this will only change the output file names
    'X_cov': ['Hispanic','Crowding','NonHisBlk','EmployService','NoHighSchool'],
    'X_control': ['PopDens','RUCA_int','Over65','Obesity'],
}


### The script will dynamically generate output file paths reflecting the options above
if config['subset']:
    if config['subset_date']:
        gtwr_model_diagnosis_path = f"gtwr_diagstats_{config['X_cov'][X_cov_run]}_[{config['bw_type']}-{config['region'] if config['subset_region'] else 'AllZcta'}-{config['month_range'][0]}_{config['month_range'][1]}].csv"
        gtwr_model_coeff_result_path = gtwr_model_diagnosis_path.replace("diagstats", "coeff")
        gtwr_model_params_path = gtwr_model_diagnosis_path.replace("diagstats", "params")
    else:
        gtwr_model_diagnosis_path = f"gtwr_diagstats_{config['X_cov'][X_cov_run]}_[{config['bw_type']}-{config['region'] if config['subset_region'] else 'AllZcta'}-AllWeeks].csv"
        gtwr_model_coeff_result_path = gtwr_model_diagnosis_path.replace("diagstats", "coeff")
        gtwr_model_params_path = gtwr_model_diagnosis_path.replace("diagstats", "params")
else:
    gtwr_model_diagnosis_path = f"gtwr_diagstats_{config['X_cov'][X_cov_run]}_[{config['bw_type']}-AllZcta-AllWeeks].csv"
    gtwr_model_coeff_result_path = gtwr_model_diagnosis_path.replace("diagstats", "coeff")
    gtwr_model_params_path = gtwr_model_diagnosis_path.replace("diagstats", "params")

print ('gtwr_model_diagnosis_path :', gtwr_model_diagnosis_path)
print ('gtwr_model_coeff_result_path :', gtwr_model_coeff_result_path)
print ('gtwr_model_params_path :', gtwr_model_params_path) 

--- 
# STEP 1:  Data Pre-Processing

In [ ]:
#############################################################################
####### STEP 1:  DATA PREP
#############################################################################

### ---------------------------------
### Prepare shapefile data 
### ---------------------------------

## Read shapefile data
if config['subset_region']:
    shape_data = gpd.read_file(config['study_area_SD'])  #Subset data with SD County only
else:
    shape_data = gpd.read_file(config['study_area_Full'])

## Process gdf, add center coordinates
shape_data = shape_data.set_crs('EPSG:4326',allow_override=True)
shape_data['Lat'] = shape_data.centroid.y
shape_data['Lon'] = shape_data.centroid.x


### ---------------------------------
### Prepare case data 
### ---------------------------------

### Read case data
cov_data = pd.read_csv(config['case_data'])

### If set in Global Parameter setting, filters data by [DateRange] and/or by [Region]
## If subset by Date Range, filter by date range (weeks)
if config['subset_date']:
    cov_data = cov_data[(cov_data['Week'] >= config['data_range_start']) & (cov_data['Week'] <= config['data_range_end'])]
    ### Example: cov_data = cov_data[(cov_data['Week'] >= '2020-04-01') & (cov_data['Week'] <= '2020-06-30')]

## If subset by region, filter case data by subset location IDs
if config['subset_region']:    
    cov_data = cov_data[cov_data['ZCTA'].isin(list(shape_data.ZCTA.unique()))]

## Sort case data by place and time (ASC)
cov_data = cov_data.sort_values(by = ['ZCTA', 'Week'], ascending = [True, True], na_position = 'first')


### -------------------------------------------------------
### Prepare some variables that needs additional care
### -------------------------------------------------------

## [RUCA]
## Recode RUCA code data to int  { All RUCA: 'Urban', 'Small Rural', 'Isolated Small Rural', 'Large Rural' }
ruca_dict = {'Urban':1,
             'Small Rural':2, 
             'Isolated Small Rural':3, 
             'Large Rural':4}

cov_data['RUCA_int'] = cov_data['RUCA'].replace(ruca_dict)

## [TIME-var] 
## Define time as a dummy variable, an incremental integer starting from 1
weeks = pd.Series(cov_data['Week'].unique()).sort_values(ascending=True)
weeks_dummy = pd.DataFrame(weeks, columns=['Week'])
weeks_dummy["time_dummy"] = np.arange(1, len(weeks_dummy) + 1)

cov_data = cov_data.merge(weeks_dummy, on='Week', how='left')

---
# STEP 2: Model Setup

In [ ]:
###############################################
####### STEP 2:  Finalize model input data
###############################################

## Join ZCTA centroid coordinates to COVID case data
cov_coords = cov_data.merge(shape_data[['ZCTA','Lat','Lon']], on='ZCTA', how='left')

### -------------------------------------------------------
### Function to standardize input variables by its own columns
### -------------------------------------------------------

from sklearn.preprocessing import StandardScaler

def standardize_col_zScore(col):
    scaler = StandardScaler()
    scaler.fit(col)
    col = scaler.transform(col)
    return col

### --------------------------------------------------------------
### Define covariate (X), control variables (C), and outcome (Y)
### --------------------------------------------------------------

## standardize covariate (X)
if config['X_cov'][X_cov_run] == 'RUCA_int' :
    X1= cov_coords[config['X_cov'][X_cov_run]].to_numpy().reshape(-1, 1)
else:
    X1=standardize_col_zScore(cov_coords[config['X_cov'][X_cov_run]].to_numpy().reshape(-1, 1))

## standardize control variables (C)
C1=standardize_col_zScore(cov_coords[config['X_control'][0]].to_numpy().reshape(-1, 1))
C2=standardize_col_zScore(cov_coords[config['X_control'][1]].to_numpy().reshape(-1, 1))
C3=standardize_col_zScore(cov_coords[config['X_control'][2]].to_numpy().reshape(-1, 1))
C4=standardize_col_zScore(cov_coords[config['X_control'][3]].to_numpy().reshape(-1, 1))

## set up all paramters for model
args = {
    'coords': cov_coords[['Lon', 'Lat']].to_numpy(),
    't': cov_coords['time_dummy'].to_numpy().reshape(-1, 1),
    'y': cov_coords['total_r'].to_numpy().reshape(-1, 1),
    'X': np.hstack([X1,C1,C2,C3,C4])
}

In [ ]:
### ###########################################################
### Define Bandwidth (bw) and spatio-temporal scale (tau) 
### option 1: use pre-defined bw/tau
### option 2: use golden search to find optimized bw/tau 
### ###########################################################

from gtwr.sel import SearchGTWRParameter
from gtwr import sel

## Optional: Search for params [bw, tau] with Golden Search
if config['use_goldenSearch']:
    sel = SearchGTWRParameter(**args, kernel = 'gaussian', fixed = True)    # fixed=True : use fixed bandwidth  |  fixed=False : use adaptive bandwidth
    bw, tau = sel.search(
        tau_min = config['tau_min'],   # use the lower-bound defined in the global parameters
        tau_max = config['tau_max'],   # use the upper-bound defined in the global parameters
        verbose = True
    )  #bw unit tau_max = 20
    print (f"Searching for best bw & tau using golden search, tau_min = {config['tau_min']} and tau_max = {config['tau_max']}")
## otherwise, use pre-defined bw/tau from the global parameter configs
else:  
    bw = config['bw']      # use the defined bandwidth 
    tau = config['tau']    # use the defined spatio-temporal scale
    print (f"Use pre-defined bw: {bw}  &  tau: {tau}")

In [ ]:
### Record the final model parameters in a dictionary, will be saved in output files later.

import pprint
pp = pprint.PrettyPrinter(indent=4)

final_settings_for_model = {
    'bw':  bw,
    'tau': tau,
    'running_with_subset': config['subset'],
    'subset_region': config['region'] if config['subset_region'] else False,
    'subset_date' : f"{config['data_range_start']}_to_{config['data_range_end']}" if config['subset_date'] else False,
    'wave' : waveID if config['subset_date'] else False,
    'X_covar': config['X_cov'][X_cov_run],
    'X_control': config['X_control']
}

print(f"GTWR model will run with the following params:")
pp.pprint(final_settings_for_model)

In [ ]:
### If indicated in the global parameters setting that this is a sensitivity test run for bw/tau,
### This step will update the output file names to reflect the bw/tau selection

if config['sensiTest_tau']:
    gtwr_model_diagnosis_path = f"{os.path.splitext(gtwr_model_diagnosis_path)[0]}_[bw-{str(bw)}_tau-{str(tau)}].csv"
    gtwr_model_coeff_result_path = f"{os.path.splitext(gtwr_model_coeff_result_path)[0]}_[bw-{str(bw)}_tau-{str(tau)}].csv"
    gtwr_model_params_path = f"{os.path.splitext(gtwr_model_params_path)[0]}_[bw-{str(bw)}_tau-{str(tau)}].csv"
print ('gtwr_model_diagnosis_path :', gtwr_model_diagnosis_path)
print ('gtwr_model_coeff_result_path :', gtwr_model_coeff_result_path)
print ('gtwr_model_params_path :', gtwr_model_params_path) 

---
# STEP 3: Fit GTWR Model

In [ ]:
################################################
### STEP 3: FIT GTWR MODEL 
################################################

gtwr = GTWR(
    **args,
    bw=bw, tau=1,
    kernel='gaussian', 
    fixed=True).fit()   # fixed=True : use fixed bandwidth  |  fixed=False : use adaptive bandwidth

In [ ]:
### Check model output: betas
# print (gtwr.betas.shape)

### Check model output: tvalues
# print (gtwr.tvalues.shape)

---
# STEP 4: Export Model Setting & Results 

In [ ]:
################################################
### STEP 4: Export Model Setting & Results 
################################################

## Save model parameters to .csv 
from datetime import datetime
final_settings_for_model['execution_date'] =  int(datetime.now().date().strftime("%Y%m%d"))
pd.DataFrame.from_dict({0:final_settings_for_model}, 'index')[['X_covar','X_control', 'running_with_subset', 'subset_date', 'subset_region','wave', 'bw','tau', 'execution_date']].to_csv(gtwr_model_params_path, index=False)


## Save model diagnostic statistics/metrics to .csv  ['AIC','BIC','R^2', Adjusted R^2']
diag_stats={
    'AIC':[gtwr.aic], 
    'BIC':[gtwr.bic],
    'R2': [gtwr.R2],
    'Adj R2':[gtwr.adj_R2]
}
diag_save=pd.DataFrame(diag_stats)
diag_save.to_csv(gtwr_model_diagnosis_path,index=False)

## Save model coefficient and t-statistic
covariates_names = [config['X_cov'][X_cov_run]] + config['X_control']

corff_col_names = ['constant_betas']
corff_col_names = corff_col_names + ['coeff_' + s for s in covariates_names]
coeff=pd.DataFrame(gtwr.betas, columns=corff_col_names)

## Save model t-statistic restuls
tStat_col_names = ['constant_tvalues']
tStat_col_names = tStat_col_names + ['tStat_' + s for s in covariates_names]
tStat=pd.DataFrame(gtwr.tvalues, columns=tStat_col_names)

## Save model  standard residual
res=pd.DataFrame(gtwr.std_res,columns=['Std_Residuals'])
final_result=cov_coords.join([coeff,tStat,res])
final_result.to_csv(gtwr_model_coeff_result_path, index=False)

print ("ALL DONE !!")